In [1]:
import tensorflow as tf

データセットを読み込む

MNIST データセットを読み込んで準備します。ここではサンプルデータを整数から浮動小数点数に変換します

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test= x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 6s 1us/step


機械学習モデルを構築

層を積み重ねてtk.keras.Sequentialモデルを構築します。訓練のためにオプティマイザと損失関数を選びます

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



各サンプルについて、モデルは "logits" または "log-odds" スコアのベクトルをクラスごとに返します。

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.37404287,  0.1738635 ,  0.7010235 , -0.28596538,  0.29389548,
        -0.53192556,  0.1974236 , -0.29049873,  0.21751311,  0.4252373 ]],
      dtype=float32)

tf.nn.softmax関数は、クラスごとにこれらのロジットを"probabilities"に変換します

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.12034305, 0.09851088, 0.1668887 , 0.06219895, 0.11107427,
        0.04863668, 0.10085937, 0.06191762, 0.10290608, 0.12666439]],
      dtype=float32)

注意: この tf.nn.softmax をネットワークの最後のレイヤーのアクティベーション関数として組み込むことも可能です。こうすることでモデルの出力をより直接的に解釈可能にすることもできますが、softmax 出力を使用する場合、すべてのモデルに対して正確で数値的に安定した損失計算を提供することは不可能であるため、この方法は推奨されません。

losses.SparseCategoricalCrossentropy 損失は、ロジットのベクトルと True インデックスを取り、各サンプルのスカラー損失を返します。

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

この損失は、True のクラスの負の対数確率に等しくなります。モデルが正しいクラスであることが確実な場合はゼロです。

トレーニングされていないこのモデルでは、ランダムに近い確率（クラス当たり 1/10）が得られるため、最初の損失は -tf.math.log(1/10) ~= 2.3 に近くなります。

In [7]:
loss_fn(y_train[:1], predictions).numpy()

3.0233772

トレーニングを開始する前に、Keras Model.compile を使用してモデルの構成とコンパイルを行います。optimizer クラスを adam に、loss を前に定義した loss_fn 関数に設定し、metrics パラメータを accuracy に設定して評価するモデルの指標を指定します

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

モデルをトレーニングして評価する<br>
損失を最小限に抑えられるようにモデルのパラメータを Model.fit メソッドで調整します。

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5


2023-04-01 10:38:14.881994: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1875/1875 [==============================] - 12s 6ms/step - loss: 0.2896 - accuracy: 0.9164
Epoch 2/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1381 - accuracy: 0.9586
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1028 - accuracy: 0.9688
Epoch 4/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0825 - accuracy: 0.9750
Epoch 5/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0695 - accuracy: 0.9781


Model.evaluateメソッドは通常、検証セットがテストセットでモデルパフォーマンスをチェックします。

In [10]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0786 - accuracy: 0.9763 - 1s/epoch - 4ms/step


[0.07862693816423416, 0.9763000011444092]

tf.nn.softmax関数はクラスごとにこれらのロジットを"確率"に変換します<br>

モデルが確率で返すようにするには、トレーニング済みのモデルをラップして、それに<br>
softmaxを接続することができます。

In [11]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[6.7872480e-07, 1.0933553e-09, 3.4200108e-05, 8.4373190e-05,
        2.3258274e-11, 1.8068009e-06, 1.3558508e-12, 9.9987376e-01,
        1.6393303e-06, 3.5846172e-06],
       [2.0656280e-07, 5.1188137e-04, 9.9941349e-01, 4.2385112e-05,
        5.6106890e-14, 5.8181040e-06, 9.0223261e-07, 6.5415872e-12,
        2.5293113e-05, 8.7589641e-10],
       [6.0043394e-06, 9.9722242e-01, 4.4351071e-04, 1.5435258e-05,
        6.2558785e-05, 3.8697508e-05, 8.9834211e-05, 1.4575985e-03,
        6.5622188e-04, 7.6980878e-06],
       [9.9999094e-01, 4.8797126e-13, 4.8989687e-06, 3.5379299e-09,
        2.1640219e-09, 4.7373831e-08, 2.0220796e-06, 6.9776922e-07,
        2.8125875e-09, 1.3498915e-06],
       [4.6314428e-05, 6.6259481e-10, 3.1578606e-05, 4.9689568e-08,
        9.8219299e-01, 1.2290172e-07, 8.8333593e-07, 4.9104512e-04,
        6.5099704e-07, 1.7236371e-02]], dtype=float32)>